# Import packages

In [18]:
import pickle

from fuzzywuzzy import process
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


# Load the dataset

In [3]:
df_facility = pd.read_csv("../data/synthetic_facility_v3.csv")
df_hdss = pd.read_csv("../data/synthetic_hdss_v3.csv")

In [4]:
df_hdss.head()

,recnr,firstname,lastname,petname,dob,sex,nationalid,hdssid,hdsshhid
0,1,Zaina,Hanifa,Ula,22-09-1930 00:00,2,NaN,I20001,HH100001
1,2,Godfrey,Maganda,Mukama,15-07-1934 00:00,1,NaN,I20002,HH100002
2,3,Kasim,Ngobi,Galabuzi,03-03-1983 00:00,1,NaN,I20003,HH100003
3,4,Esther,NaN,Inara,30-07-1968 00:00,2,NaN,I20004,HH100004
4,5,Sumaya,Swabula,NaN,13-12-1930 00:00,2,NaN,I20005,HH100005


In [5]:
df_facility.head()

,recnr,firstname,lastname,petname,dob,sex,nationalid,patientid,visitdate
0,2,Fatuma,NaN,Zaina,24-08-2017 00:00,2,N_ID_5000,2069,10-09-2018
1,3,Gloria,Rashida,NaN,11-07-1993 00:00,2,N_ID_11861,2079,14-12-2022
2,4,Ali,Hakram,Igomu,17-05-2014 00:00,1,N_ID_11864,2080,09-06-2023
3,5,Nakalema,NaN,Nkwanga,27-02-2026 00:00,2,N_ID_11867,2081,07-02-2019
4,6,Asuman,Sempa,Aguti,02-03-2002 00:00,1,N_ID_11870,2082,18-08-2020


# Resize datasets to include only the related columns

In [7]:
def find_similar_columns(df1, df2, threshold=80):
    similar_columns = {}

    for col1 in df1.columns:
        match, score = process.extractOne(col1, df2.columns)
        if score >= threshold:
            similar_columns[col1] = match

    return similar_columns

In [8]:
# Get the similar colums with a threshold of 80
similar_columns = find_similar_columns(df_facility, df_hdss, threshold=80)

# Resize the dataframes to include only the similar columns
df_hdss_resized = df_hdss[list(similar_columns.values())]
df_facility_resized = df_facility[list(similar_columns.keys())]

In [9]:
df_hdss_resized.head()

,recnr,firstname,lastname,petname,dob,sex,nationalid
0,1,Zaina,Hanifa,Ula,22-09-1930 00:00,2,NaN
1,2,Godfrey,Maganda,Mukama,15-07-1934 00:00,1,NaN
2,3,Kasim,Ngobi,Galabuzi,03-03-1983 00:00,1,NaN
3,4,Esther,NaN,Inara,30-07-1968 00:00,2,NaN
4,5,Sumaya,Swabula,NaN,13-12-1930 00:00,2,NaN


In [10]:
df_facility_resized.head()

,recnr,firstname,lastname,petname,dob,sex,nationalid
0,2,Fatuma,NaN,Zaina,24-08-2017 00:00,2,N_ID_5000
1,3,Gloria,Rashida,NaN,11-07-1993 00:00,2,N_ID_11861
2,4,Ali,Hakram,Igomu,17-05-2014 00:00,1,N_ID_11864
3,5,Nakalema,NaN,Nkwanga,27-02-2026 00:00,2,N_ID_11867
4,6,Asuman,Sempa,Aguti,02-03-2002 00:00,1,N_ID_11870


# Similarity with TF-IDF Vectorization

In [15]:
df_hdss['combined'] = df_hdss_resized.apply(lambda row: " ".join(row.astype(str)), axis=1)
df_facility['combined'] = df_facility_resized.apply(lambda row: " ".join(row.astype(str)), axis=1)

In [16]:
df_hdss.head()

,recnr,firstname,lastname,petname,dob,sex,nationalid,hdssid,hdsshhid,combined
0,1,Zaina,Hanifa,Ula,22-09-1930 00:00,2,NaN,I20001,HH100001,1 Zaina Hanifa Ula 22-09-1930 00:00 2 nan
1,2,Godfrey,Maganda,Mukama,15-07-1934 00:00,1,NaN,I20002,HH100002,2 Godfrey Maganda Mukama 15-07-1934 00:00 1 nan
2,3,Kasim,Ngobi,Galabuzi,03-03-1983 00:00,1,NaN,I20003,HH100003,3 Kasim Ngobi Galabuzi 03-03-1983 00:00 1 nan
3,4,Esther,NaN,Inara,30-07-1968 00:00,2,NaN,I20004,HH100004,4 Esther nan Inara 30-07-1968 00:00 2 nan
4,5,Sumaya,Swabula,NaN,13-12-1930 00:00,2,NaN,I20005,HH100005,5 Sumaya Swabula nan 13-12-1930 00:00 2 nan


In [17]:
df_facility.head()

,recnr,firstname,lastname,petname,dob,sex,nationalid,patientid,visitdate,combined
0,2,Fatuma,NaN,Zaina,24-08-2017 00:00,2,N_ID_5000,2069,10-09-2018,2 Fatuma nan Zaina 24-08-2017 00:00 2 N_ID_5000
1,3,Gloria,Rashida,NaN,11-07-1993 00:00,2,N_ID_11861,2079,14-12-2022,3 Gloria Rashida nan 11-07-1993 00:00 2 N_ID_1...
2,4,Ali,Hakram,Igomu,17-05-2014 00:00,1,N_ID_11864,2080,09-06-2023,4 Ali Hakram Igomu 17-05-2014 00:00 1 N_ID_11864
3,5,Nakalema,NaN,Nkwanga,27-02-2026 00:00,2,N_ID_11867,2081,07-02-2019,5 Nakalema nan Nkwanga 27-02-2026 00:00 2 N_ID...
4,6,Asuman,Sempa,Aguti,02-03-2002 00:00,1,N_ID_11870,2082,18-08-2020,6 Asuman Sempa Aguti 02-03-2002 00:00 1 N_ID_1...


In [19]:
# Vectorized the combined column using TF-IDF
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit(df_hdss['combined'])

# Save the vectorizer for future use
with open('../ml-models/tf-idf-vectorizer.pkl', 'wb') as file:
    pickle.dump(vectorizer, file)

In [46]:
def find_closest_match_id_tf_idf(string_row, loaded_vectorizer, vectors, threshold=0.7):
   # vectorize the combined (stringified) row
   new_vector = loaded_vectorizer.transform([string_row])

   # compute similarity btn the new vector and the hdss dataset vectors
   similarity_scores = cosine_similarity(new_vector, vectors)

   # Find the index of the closest match
   closest_match_index = np.argmax(similarity_scores)

   # Check if the highest similarity score is above the threshold
   if similarity_scores[0, closest_match_index] >= threshold:
      return closest_match_index
   else:
      return None

In [47]:
 # load tf_idf_vectorizer
with open('../ml-models/tf-idf-vectorizer.pkl', 'rb') as file:
    loaded_vectorizer = pickle.load(file)

vectors = loaded_vectorizer.transform(df_hdss['combined'])

# testing with 10 rows in the facility dataset
for index, row in df_facility.iloc[0:10, :].iterrows():
    # print(row)
    combined_row_string = row['combined']
    closest_match_id = find_closest_match_id_tf_idf(combined_row_string, loaded_vectorizer, vectors)

    # check if the closest match is a valid id
    if closest_match_id:
        closest_match = df_hdss.iloc[closest_match_id]
        print(f"Closest_match: {closest_match['combined']} - Matched with {combined_row_string}")
    else:
        print(f"Record: {combined_row_string} has no appropriate match")


Closest_match: 2741 Fatuma Zaina nan 24-08-2017 00:00 2 nan - Matched with 2 Fatuma nan Zaina 24-08-2017 00:00 2 N_ID_5000
Closest_match: 2613 Gloria Rashida nan 11-07-1993 00:00 2 nan - Matched with 3 Gloria Rashida nan 11-07-1993 00:00 2 N_ID_11861
Record: 4 Ali Hakram Igomu 17-05-2014 00:00 1 N_ID_11864 has no appropriate match
Record: 5 Nakalema nan Nkwanga 27-02-2026 00:00 2 N_ID_11867 has no appropriate match
Record: 6 Asuman Sempa Aguti 02-03-2002 00:00 1 N_ID_11870 has no appropriate match
Closest_match: 1366 Haifa Alice Xama 20-12-1945 00:00 2 nan - Matched with 7 Haifa Alice Xama 27-12-1945 00:00 2 N_ID_11873
Closest_match: 2902 Namutamba Sumaya Bria 18-02-1997 00:00 2 nan - Matched with 8 Namutamba Sumaya Bria 18-02-1997 00:00 2 N_ID_11876
Closest_match: 2951 Madina Shirati Sira 10-07-1997 00:00 2 nan - Matched with 9 Madina Shirati Sira 10-07-1997 00:00 2 N_ID_5006
Record: 10 Mulondo Kaziba Anko 11-11-1976 00:00 1 N_ID_5009 has no appropriate match
Record: 11 Bakali Kisambi